In [ ]:
import autogen
import os
from xentropy.tool import Tool

os.environ['AZURE_OPENAI_API_BASE'] = 'YOUR_AZURE_OPENAI_API_BASE'
os.environ['AZURE_OPENAI_KEY'] = 'YOUR_AZURE_OPENAI_KEY'
os.environ['XENTROPY_API_KEY'] = 'YOUR_XENTROPY_API_KEY'

In [ ]:
# Get a tool that converts address to latitude longitude coordinate
geocoding = Tool.load('xentropy--geocoding', api_key=os.environ.get('XENTROPY_API_KEY'))
# Get a tool that computes the earth surface distance between two coordinates
geodesic = Tool.load('xentropy--geodesic', api_key=os.environ.get('XENTROPY_API_KEY'))


In [ ]:
config_list = autogen.config_list_from_models(model_list=[os.environ.get('AZURE_OPENAI_DEPLOYMENT')]) # your choice of LLM

llm_config = {
    "functions": [
        {
            "name": geocoding.name,
            "description": geocoding.description,
            "parameters": geocoding.input_json_schema,
        },
        {
            "name": geodesic.name,
            "description": geodesic.description,
            "parameters": geodesic.input_json_schema,
        },
    ],
    "config_list": config_list,
    "timeout": 120,
}

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="Use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "") == ""
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get(
        'role') == 'user' and x.get('content') != None,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    function_map={
        geocoding.name: geocoding.run,
        geodesic.name: geodesic.run,
    }
)

In [ ]:
# Notice that in order to calculate the distance. We need to first convert the address to geolocation,
# and then calculate the distance between the two geolocations. 
# LLM Agents can composite two functions to achieve this goal.
user_proxy.initiate_chat(
    chatbot,
    message="What is the distance between Gare Port La Goulette - Sud in Tunisia and Porto di Napoli in Italy?",
)